In [12]:
import pandas as pd
from datetime import datetime
import os
import re

customer_df = pd.read_csv("../data/raw/customer_data.csv")
retail_df = pd.read_csv("../data/raw/retail_data.csv")


# Cleaning

Customer

In [13]:
# --- 1. Normalizar nombres de columnas ---
customer_df.columns = customer_df.columns.str.strip().str.lower()
print("Nombres de columnas normalizados.")

# --- 2. signup_date: validar rango y corregir ---
customer_df['signup_date'] = pd.to_datetime(customer_df['signup_date'], errors='coerce')

min_signup = pd.Timestamp("2020-01-01")
max_signup = pd.Timestamp.today().normalize()

# Reemplazar inválidas por la fecha más reciente
customer_df['signup_date'] = customer_df['signup_date'].apply(
    lambda x: max_signup if pd.isna(x) or x < min_signup or x > max_signup else x
)

print("signup_date limpiado y validado correctamente.")

# --- 3. email: limpiar, validar y asegurar unicidad ---
email_regex = (
    r"^(?!\.)[A-Za-z0-9._%+-]+@"
    r"(?!-)[A-Za-z0-9-]+(\.[A-Za-z0-9-]+)*\.[A-Za-z]{2,}$"
)

def fix_email(row):
    email = str(row['email']).strip()
    if pd.isna(email) or not re.match(email_regex, email):
        return f"unknown_{row['id']}@domain.com"
    return email

customer_df['email'] = customer_df.apply(fix_email, axis=1)

# Resolver duplicados agregando índice
dup_emails = customer_df.duplicated(subset=['email'], keep=False)
if dup_emails.any():
    print(f"Se encontraron {dup_emails.sum()} emails duplicados, corrigiendo...")
    customer_df.loc[dup_emails, 'email'] = (
        customer_df.loc[dup_emails, 'email'] + "_" + customer_df.loc[dup_emails].index.astype(str)
    )

print("email limpiado, validado y sin duplicados correctamente.")

# --- 4. phone: limpiar, corregir y asegurar unicidad ---
customer_df['phone'] = customer_df['phone'].astype(str).str.replace(r'\D', '', regex=True)

# Identificar números inválidos (distintos de 10 dígitos)
invalid_mask = customer_df['phone'].apply(lambda x: len(x) != 10)

if invalid_mask.any():
    invalid_count = invalid_mask.sum()
    print(f"Se encontraron {invalid_count} teléfonos inválidos. Asignando números consecutivos...")

    # Crear nuevos números consecutivos de 10 dígitos
    new_phones = [str(i).zfill(10) for i in range(invalid_count)]
    customer_df.loc[invalid_mask, 'phone'] = new_phones

# Asegurar unicidad agregando sufijo si quedan duplicados
dup_phones = customer_df.duplicated(subset=['phone'], keep=False)
if dup_phones.any():
    print(f"Se encontraron {dup_phones.sum()} teléfonos duplicados. Corrigiendo...")
    customer_df.loc[dup_phones, 'phone'] = (
        customer_df.loc[dup_phones, 'phone'] + "_" + customer_df.loc[dup_phones].index.astype(str)
    )

print("phone limpiado y validado correctamente.")

# --- 5. age: forzar rango 18-100 ---
customer_df['age'] = pd.to_numeric(customer_df['age'], errors='coerce').fillna(18).astype(int)
customer_df['age'] = customer_df['age'].clip(lower=18, upper=100)

print("age limpiado y validado correctamente.")

# --- 6. gender: normalizar y validar ---
customer_df['gender'] = customer_df['gender'].astype(str).str.strip().str.capitalize()
valid_genders = ["Male", "Female", "M", "F", "Other"]
customer_df['gender'] = customer_df['gender'].apply(lambda x: x if x in valid_genders else "Other")

print("gender limpiado y normalizado correctamente.")

# --- 7. address y full_name: llenar nulos ---
customer_df['address'].fillna("Unknown Address", inplace=True)
customer_df['full_name'].fillna("Unknown", inplace=True)

print("address y full_name limpiados correctamente.")


Nombres de columnas normalizados.
signup_date limpiado y validado correctamente.
email limpiado, validado y sin duplicados correctamente.
Se encontraron 314 teléfonos inválidos. Asignando números consecutivos...
phone limpiado y validado correctamente.
age limpiado y validado correctamente.
gender limpiado y normalizado correctamente.
address y full_name limpiados correctamente.


Retail

In [14]:
# --- 1. transaction_id: asegurar unicidad sin eliminar ---
retail_df['transaction_id'] = retail_df['transaction_id'].astype(str).str.strip()
retail_df['transaction_id'] = retail_df['transaction_id'].replace(['', 'nan', 'None'], pd.NA)

# Paso 1: obtener máximo ID actual
max_id = pd.to_numeric(retail_df['transaction_id'], errors='coerce').max(skipna=True)
if pd.isna(max_id):
    max_id = 0

# Paso 2: asignar IDs a los nulos
missing_count = retail_df['transaction_id'].isna().sum()
if missing_count > 0:
    new_ids = range(int(max_id) + 1, int(max_id) + missing_count + 1)
    retail_df.loc[retail_df['transaction_id'].isna(), 'transaction_id'] = [str(i) for i in new_ids]
    print(f"Se asignaron {missing_count} nuevos transaction_id.")

# Paso 3: corregir duplicados con IDs consecutivos
duplicados = retail_df.duplicated(subset=["transaction_id"], keep="first")
if duplicados.any():
    dup_count = duplicados.sum()
    print(f"Se encontraron {dup_count} duplicados en 'transaction_id'. Corrigiendo...")

    max_id = pd.to_numeric(retail_df['transaction_id'], errors='coerce').max()
    new_dup_ids = range(int(max_id) + 1, int(max_id) + dup_count + 1)
    retail_df.loc[duplicados, 'transaction_id'] = [str(i) for i in new_dup_ids]

print("transaction_id corregido, sin eliminar registros.")

# --- 2. purchase_date: validar y ajustar rango ---
retail_df['purchase_date'] = pd.to_datetime(retail_df['purchase_date'], errors='coerce')

min_date = pd.Timestamp("2025-01-01")
max_date = pd.Timestamp.today().normalize()

retail_df['purchase_date'] = retail_df['purchase_date'].apply(
    lambda x: min_date if pd.isna(x) or x < min_date else (max_date if x > max_date else x)
)

print("purchase_date limpiado y validado correctamente.")

# --- 3. amount: convertir a float y limitar rango ---
retail_df['amount'] = pd.to_numeric(retail_df['amount'], errors='coerce')
retail_df['amount'].fillna(0.01, inplace=True)
retail_df['amount'] = retail_df['amount'].clip(lower=0.01, upper=10000)

print("amount limpiado y validado correctamente.")

# --- 4. product_category: llenar nulos y vacíos ---
retail_df['product_category'].fillna('No specified', inplace=True)
retail_df['product_category'].replace("", "No specified", inplace=True)

print("product_category limpiado correctamente.")

Se encontraron 720 duplicados en 'transaction_id'. Corrigiendo...
transaction_id corregido, sin eliminar registros.
purchase_date limpiado y validado correctamente.
amount limpiado y validado correctamente.
product_category limpiado correctamente.


In [15]:
print("\n--- Customer Data ---")
print(customer_df.info())
print("\nNulos por columna:")
print(customer_df.isnull().sum())

print("\n--- Retail Data ---")
print(retail_df.info())
print("\nNulos por columna:")
print(retail_df.isnull().sum())

print("\nLimpieza completada correctamente.")


--- Customer Data ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           1000 non-null   int64         
 1   full_name    1000 non-null   object        
 2   email        1000 non-null   object        
 3   phone        1000 non-null   object        
 4   address      1000 non-null   object        
 5   signup_date  1000 non-null   datetime64[ns]
 6   name         1000 non-null   object        
 7   gender       1000 non-null   object        
 8   age          1000 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 70.4+ KB
None

Nulos por columna:
id             0
full_name      0
email          0
phone          0
address        0
signup_date    0
name           0
gender         0
age            0
dtype: int64

--- Retail Data ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8

In [16]:
customer_df.reset_index(drop=True, inplace=True)
retail_df.reset_index(drop=True, inplace=True)

# Crear carpeta clean si no existe
os.makedirs("../data/clean", exist_ok=True)

# Rutas de guardado
customer_clean_path = "../data/clean/customer_data_clean.csv"
retail_clean_path = "../data/clean/retail_data_clean.csv"

# Guardar los CSV limpios
customer_df.to_csv(customer_clean_path, index=False)
retail_df.to_csv(retail_clean_path, index=False)

print(f"Cleaned customer data saved at: {customer_clean_path}")
print(f"Cleaned retail data saved at: {retail_clean_path}")

Cleaned customer data saved at: ../data/clean/customer_data_clean.csv
Cleaned retail data saved at: ../data/clean/retail_data_clean.csv
